<a href="https://colab.research.google.com/github/Jacob-McM/Neural-Network-Modelling/blob/main/AlphabetSoupCharity_Optimzation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
## Upload File
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data (1).csv


In [3]:
#  Import and read the charity_data.csv.
import pandas as pd 
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
application_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
## Drop Identification Columns 
application_df = application_df.drop(columns=['EIN', 'NAME'])

APPLICATION_TYPE column

In [5]:
## Make workable dataframe out of `APPLICAITON_TYPE` column to use functions on to determine values and potential cut-off points & bins

app_counts = pd.DataFrame()
app_counts['count'] = application_df['APPLICATION_TYPE'].value_counts()
app_sum_counts = application_df['APPLICATION_TYPE'].value_counts().sum()
print(application_df['APPLICATION_TYPE'].value_counts())
print(f"Sum of all values: {application_df['APPLICATION_TYPE'].value_counts().sum()}")

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
Sum of all values: 34299


In [6]:
app_counts['percentage'] = app_counts['count']/app_sum_counts*100
app_counts.reset_index(inplace=True)

In [7]:
## Decided cut off will be app types that cover less than 1%. 
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

CLASSIFICATION_TYPE column

In [8]:
## Make workable dataframe out of `CLASSIFICATION` column to use functions on to determine values and potential cut-off points & bins

class_counts = pd.DataFrame()
class_counts['count'] = application_df['CLASSIFICATION'].value_counts()
class_sum_counts = application_df['CLASSIFICATION'].value_counts().sum()

print(application_df['CLASSIFICATION'].value_counts())
print(f"Sum of all values: {application_df['CLASSIFICATION'].value_counts().sum()}")

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
Sum of all values: 34299


In [9]:
class_counts['percentage'] = class_counts['count']/class_sum_counts*100
class_cutoff = class_counts.loc[class_counts['count'] <= 100].copy()
class_cutoff = class_cutoff.reset_index()
class_cutoff

,index,count,percentage
0,C2800,95,0.276976
1,C7100,75,0.218665
2,C1300,58,0.169101
3,C1280,50,0.145777
4,C1230,36,0.104959
5,C1400,34,0.099128
6,C7200,32,0.093297
7,C2300,32,0.093297
8,C1240,30,0.087466
9,C8000,20,0.058311


In [10]:
## ADJUSTED Decided cut-off will be classification types with less than a count of 100

classifications_to_replace = class_cutoff['index'].values.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
# application_df['CLASSIFICATION'].value_counts()
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

Data type conversion

In [11]:
## Convert to numerical data
preprocess_df = pd.get_dummies(application_df)
preprocess_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
## Define Target values (y) and Features (X)
y = preprocess_df['IS_SUCCESSFUL']
X = preprocess_df.drop('IS_SUCCESSFUL', axis=1).values

## Split data in training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =7)

In [13]:
## Scaler instance
scaler = StandardScaler()

## Fit StandardScaler
X_scaler = scaler.fit(X_train)

## Scale Data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Attempt 2

With new cut-off for classifications

In [14]:
input_features = len(X_train[0])
hidden_layer_1 = 36
hidden_layer_2 = 26
outputs = 1

nn = tf.keras.models.Sequential()

## First Hidden Layer
nn.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Output Node
nn.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 36)                1800      
                                                                 
 dense_1 (Dense)             (None, 26)                962       
                                                                 
 dense_2 (Dense)             (None, 1)                 27        
                                                                 
Total params: 2,789
Trainable params: 2,789
Non-trainable params: 0
_________________________________________________________________


In [15]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5782 - accuracy: 0.7150
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7302
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7313
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7319
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7322
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5434 - accura

In [16]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5578 - accuracy: 0.7282 - 528ms/epoch - 2ms/step
Loss: 0.5577719211578369, Accuracy: 0.7281632423400879


## Attempt 3 
Using different parameters and different activation function ELU

In [17]:
input_features = len(X_train[0])
hidden_layer_1 = 36
hidden_layer_2 = 26
outputs = 1

nn_3 = tf.keras.models.Sequential()

## First Hidden Layer
nn_3.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='elu'))

## Second Hidden Layer
nn_3.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='elu'))

## Output Node
nn_3.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 36)                1800      
                                                                 
 dense_4 (Dense)             (None, 26)                962       
                                                                 
 dense_5 (Dense)             (None, 1)                 27        
                                                                 
Total params: 2,789
Trainable params: 2,789
Non-trainable params: 0
_________________________________________________________________


In [18]:
nn_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5800 - accuracy: 0.7134
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7292
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7302
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7310
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accura

In [19]:
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5551 - accuracy: 0.7262 - 490ms/epoch - 2ms/step
Loss: 0.5550565123558044, Accuracy: 0.7261807322502136


# Attempt 4
Using RELU again, increasing epochs

In [20]:
input_features = len(X_train[0])
hidden_layer_1 = 36
hidden_layer_2 = 26
outputs = 1

nn_4 = tf.keras.models.Sequential()

## First Hidden Layer
nn_4.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn_4.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Output Node
nn_4.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn_4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 36)                1800      
                                                                 
 dense_7 (Dense)             (None, 26)                962       
                                                                 
 dense_8 (Dense)             (None, 1)                 27        
                                                                 
Total params: 2,789
Trainable params: 2,789
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn_4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_4.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5738 - accuracy: 0.7191
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7283
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7307
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7307
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7318
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7323
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5460 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accuracy: 0.7330
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7333
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5440 - accura

In [22]:
model_loss, model_accuracy = nn_4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5711 - accuracy: 0.7268 - 504ms/epoch - 2ms/step
Loss: 0.5710723400115967, Accuracy: 0.7267638444900513


## Attempt 5 
Using more Epochs(200) and adjusted hidden layers

In [23]:
input_features = len(X_train[0])
hidden_layer_1 = 40
hidden_layer_2 = 30
outputs = 1

nn_5 = tf.keras.models.Sequential()

## First Hidden Layer
nn_5.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn_5.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Output Node
nn_5.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn_5.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 40)                2000      
                                                                 
 dense_10 (Dense)            (None, 30)                1230      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3,261
Trainable params: 3,261
Non-trainable params: 0
_________________________________________________________________


In [24]:
nn_5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_5.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5705 - accuracy: 0.7210
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7290
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7300
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7315
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7336
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7332
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5450 - accuracy: 0.7325
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5446 - accuracy: 0.7345
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7345
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5430 - accura

In [25]:
model_loss, model_accuracy = nn_5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5719 - accuracy: 0.7259 - 531ms/epoch - 2ms/step
Loss: 0.571858823299408, Accuracy: 0.7259474992752075


## Attempt 6
This attempt will use 500 epochs and the adjusted hidden layer nodes. 

In [26]:
input_features = len(X_train[0])
hidden_layer_1 = 40
hidden_layer_2 = 30
outputs = 1

nn_6 = tf.keras.models.Sequential()

## First Hidden Layer
nn_6.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn_6.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Output Node
nn_6.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn_6.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 40)                2000      
                                                                 
 dense_13 (Dense)            (None, 30)                1230      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3,261
Trainable params: 3,261
Non-trainable params: 0
_________________________________________________________________


In [27]:
nn_6.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_6.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500
804/804 [==============================] - 3s 2ms/step - loss: 0.5760 - accuracy: 0.7131
Epoch 2/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7296
Epoch 3/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7297
Epoch 4/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7324
Epoch 5/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5464 - accuracy: 0.7321
Epoch 6/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7324
Epoch 7/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accuracy: 0.7334
Epoch 8/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5440 - accuracy: 0.7331
Epoch 9/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7338
Epoch 10/500
804/804 [==============================] - 2s 2ms/step - loss: 0.5425 - accura

In [28]:
model_loss, model_accuracy = nn_6.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5851 - accuracy: 0.7268 - 462ms/epoch - 2ms/step
Loss: 0.5851298570632935, Accuracy: 0.7267638444900513


Attempt 7, Final Attempt

Reduce epochs back 200, add hidden layer

In [29]:
input_features = len(X_train[0])
hidden_layer_1 = 40
hidden_layer_2 = 30
hidden_layer_3 = 20
outputs = 1

nn_7 = tf.keras.models.Sequential()

## First Hidden Layer
nn_7.add(tf.keras.layers.Dense(units = hidden_layer_1, input_dim = input_features, activation='relu'))

## Second Hidden Layer
nn_7.add(tf.keras.layers.Dense(units=hidden_layer_2, activation='relu'))

## Third Hidden Layer
nn_7.add(tf.keras.layers.Dense(units=hidden_layer_3, activation='relu'))

## Output Node
nn_7.add(tf.keras.layers.Dense(units=outputs, activation='sigmoid'))

nn_7.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 40)                2000      
                                                                 
 dense_16 (Dense)            (None, 30)                1230      
                                                                 
 dense_17 (Dense)            (None, 20)                620       
                                                                 
 dense_18 (Dense)            (None, 1)                 21        
                                                                 
Total params: 3,871
Trainable params: 3,871
Non-trainable params: 0
_________________________________________________________________


In [32]:
nn_7.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_model = nn_7.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5740 - accuracy: 0.7164
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7296
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7311
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7312
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7335
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5449 - accuracy: 0.7334
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accuracy: 0.7334
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5438 - accuracy: 0.7351
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accuracy: 0.7348
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accura

In [33]:
model_loss, model_accuracy = nn_7.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5798 - accuracy: 0.7254 - 482ms/epoch - 2ms/step
Loss: 0.57977694272995, Accuracy: 0.7253644466400146


In [34]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
tf.keras.Model.save(nn_7,filepath='AlphabetSoupCharity_Optimization.h5')